In [34]:
import pandas as pd

# Definisci il percorso del file parquet
# Assicurati che 'artgraph_metadata.parquet' sia nella stessa directory del notebook
METADATA_FILE_PATH = 'artgraph_metadata.parquet'

print(f"Tentativo di caricare il file metadati: {METADATA_FILE_PATH}")

try:
    # Carica il file parquet in un DataFrame Pandas
    metadata_df = pd.read_parquet(METADATA_FILE_PATH)
    
    print("\nMetadati caricati con successo!")
    
    # Stampa le prime 3 righe del DataFrame
    print("\n--- Prime 3 righe dei metadati ---")
    print(metadata_df.head(3))
    
    # Stampa i nomi di tutte le colonne disponibili
    print("\n--- Nomi delle colonne disponibili nei metadati ---")
    print(metadata_df.columns.tolist())
    
    # Stampa il numero totale di righe e colonne
    print(f"\nDimensioni del DataFrame: {metadata_df.shape[0]} righe, {metadata_df.shape[1]} colonne.")
    
except FileNotFoundError:
    print(f"\nERRORE: Il file '{METADATA_FILE_PATH}' non è stato trovato.")
    print("Assicurati che il file si trovi nella stessa directory del tuo notebook.")
except Exception as e:
    print(f"\nERRORE inatteso durante il caricamento dei metadati: {e}")

print("\nEsplorazione metadati completata.")

Tentativo di caricare il file metadati: artgraph_metadata.parquet

Metadati caricati con successo!

--- Prime 3 righe dei metadati ---
                        ArtworkTitle            ArtistName ArtworkYear Period  \
0  Menton.&#160;Beach with umbrellas  zinaida-serebriakova        1931   None   
1                        Autumn Song                  erte        None   None   
2                           Feathers                  erte        None   None   

      Style                                           FileName  \
0  art deco  zinaida-serebriakova_menton-beach-with-umbrell...   
1  art deco                               erte_autumn-song.jpg   
2  art deco                                  erte_feathers.jpg   

               Genre                                           Movement  
0     genre painting  Mir iskusstva, Neoclassical architecture, Repr...  
1  symbolic painting                                               None  
2             design                                 

In [35]:
import os
import base64
from pathlib import Path
from PIL import Image
import pandas as pd
from ollama import chat
from io import BytesIO
import re # Necessario per la tokenizzazione delle frasi
import numpy as np # Necessario per np.nan in statistiche future
import torch # Import aggiunto per torch.cuda.empty_cache()
import csv

# --- CONFIGURAZIONE GLOBALE ---
# Percorso alla directory delle immagini
# Assicurati che 'images100' sia la cartella principale contenente le immagini.
IMAGE_SOURCE_DIR = './images100' 

# Modello Ollama da utilizzare
OLLAMA_MODEL = 'qwen2.5vl'

# Limita l'elaborazione alle prime N immagini (metti None per processare tutte le immagini)
IMAGES_TO_PROCESS = None # Imposta a None per processare tutte le immagini

# Numero di frammenti per la griglia 2x2
GRID_ROWS = 2
GRID_COLS = 2
TOTAL_GRID_SEGMENTS = GRID_ROWS * GRID_COLS # Sarà 4

# Numero massimo di token da generare per le descrizioni (modifica qui manualmente per i tuoi test)
MAX_TOKENS = 512 

# Percorso del file metadati
METADATA_FILE_PATH = 'artgraph_metadata.parquet'

# Nome del file CSV di output della Cella 2
MAIN_OUTPUT_CSV_FILENAME = f"analisi_immagini_qwen_2x2_con_metadati_max_tokens_{MAX_TOKENS}.csv"

# Temperatura per la generazione del modello (valori più alti = più creatività, valori più bassi = più deterministico)
TEMPERATURE = 0.7 # <--- NUOVA VARIABILE AGGIUNTA QUI

# --- PROMPT ADATTATI PER LE IMMMAGINI ---
# Questo prompt è progettato per dare priorità assoluta alla descrizione del frammento,
# usando i metadati come contesto informativo secondario, se disponibili.
PROMPT_FRAGMENT_BASE = (
    "Descrivi in modo conciso ma dettagliato **solo ciò che è visibile in questa specifica porzione dell'immagine.** "
    "Focalizzati attentamente sugli elementi, soggetti, colori, texture, forme e sulla composizione presenti nel frammento. "
    "Non fare inferenze o aggiungere dettagli non direttamente osservabili qui. "
    "Fornisci la tua descrizione in italiano."
)
# Aggiunta una variabile per il contesto metadato che verrà appeso alla fine del prompt base
METADATA_CONTEXT_SUFFIX = (
    " (Contesto: questa porzione fa parte di {metadata_info})."
)

# Nuovo prompt per l'analisi totale dell'immagine
PROMPT_TOTAL_IMAGE_BASE = (
    "Analizza e descrivi l'intera immagine nel suo complesso. "
    "Focalizzati su temi generali, composizione complessiva, interazione tra gli elementi, "
    "e il messaggio o l'emozione che l'opera trasmette. "
    "Non limitarti ai dettagli di singoli frammenti ma offri una visione olistica. "
    "Fornisci la tua descrizione in italiano."
)


# --- FUNZIONI DI UTILITY ---

def encode_image_to_base64(image: Image.Image, size=(256, 256)) -> str:
    """
    Ridimensiona un'immagine PIL e la codifica in base64.
    """
    try:
        img_resized = image.resize(size) 
        
        buffer = BytesIO()
        img_resized.save(buffer, format="JPEG") 
        img_bytes = buffer.getvalue()
        
        return base64.b64encode(img_bytes).decode('utf-8')
    except Exception as e:
        print(f"Errore nella codifica dell'immagine: {e}")
        return None

def split_image_into_grid_segments(image: Image.Image, rows: int, cols: int) -> list[Image.Image]:
    """
    Divide un'immagine PIL in una griglia di segmenti (e.g., 2x2).
    Restituisce i segmenti in ordine di lettura (da sinistra a destra, dall'alto in basso).
    """
    width, height = image.size
    segment_width = width // cols
    segment_height = height // rows
    segments = []
    
    for r in range(rows):
        for c in range(cols):
            left = c * segment_width
            upper = r * segment_height
            right = (c + 1) * segment_width if c < cols - 1 else width
            lower = (r + 1) * segment_height if r < rows - 1 else height
            
            segment = image.crop((left, upper, right, lower))
            segments.append(segment)
            
    return segments

def generate_description_ollama(model_name: str, image_base64: str, prompt: str) -> str:
    """
    Genera una descrizione usando un modello Ollama specifico con un'immagine in base64.
    """
    try:
        response = chat(
            model=model_name,
            messages=[
                {
                    'role': 'user',
                    'content': prompt,
                    'images': [image_base64],
                }
            ],
            options={
                'num_predict': MAX_TOKENS,
                'temperature': TEMPERATURE # Ora TEMPERATURE è definita globalmente
            }
        )
        return response.message.content.strip()
    except Exception as e:
        return f"Errore nell'inferenza con {model_name}: {e}"

# --- VERIFICA DIRECTORY IMMAGINI ---
if not os.path.isdir(IMAGE_SOURCE_DIR):
    print(f"ERRORE: La directory delle immagini '{IMAGE_SOURCE_DIR}' non è stata trovata.")
    print("Assicurati che la cartella 'images100' sia nella stessa directory del notebook e contenga le tue immagini.")

print("Setup completato e funzioni utility caricate.")

Setup completato e funzioni utility caricate.


In [36]:
# Cella 3: Generazione Descrizioni Immagini con Ollama e Metadati

print(f"\n\n--- INIZIO ANALISI IMMAGINI con {OLLAMA_MODEL} (via Ollama) ---")
print(f"  MAX_TOKENS impostato a: {MAX_TOKENS}")

# Questa lista raccoglierà i dizionari per ogni riga del CSV finale
all_image_descriptions_data = []
processed_images_count = 0

# --- Caricamento e preparazione dei metadati ---
metadata_df = None
metadata_dict = {} # Dizionario per lookup veloce
try:
    # Usa METADATA_FILE_PATH come definito nella Cella 2
    metadata_df = pd.read_parquet(METADATA_FILE_PATH, engine='pyarrow') 
    print(f"\nMetadati '{METADATA_FILE_PATH}' caricati con successo.")
    
    # PULIZIA INIZIALE E AGGRESSIVA DEI METADATI APPENA CARICATI QUI
    # Applica la pulizia a tutte le colonne di tipo 'object' (stringa)
    for col in metadata_df.select_dtypes(include=['object']).columns:
        # Sostituisci newline, ritorni a capo e &#160; con spazi, poi togli spazi extra ai bordi
        metadata_df[col] = metadata_df[col].astype(str).str.replace('\n', ' ').str.replace('\r', ' ').str.replace('&#160;', ' ').str.strip()
        # Rimuovi eventuali spazi doppi per pulizia ulteriore
        metadata_df[col] = metadata_df[col].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

    # Prepara un dizionario per una ricerca efficiente usando 'FileName' come chiave
    metadata_dict = metadata_df.set_index('FileName').to_dict('index')
    print("Metadati preparati per la ricerca rapida tramite 'FileName' e puliti da caratteri speciali.")

except FileNotFoundError:
    print(f"\nAVVISO: File '{METADATA_FILE_PATH}' non trovato. Le descrizioni non saranno arricchite con metadati.")
    metadata_df = None
except Exception as e:
    print(f"\nERRORE nel caricamento/preparazione dei metadati: {e}. Le descrizioni non saranno arricchite con metadati.")
    metadata_df = None
# --- FINE Caricamento Metadati ---

# Ottieni la lista dei file immagine dalla directory delle immagini (IMAGE_SOURCE_DIR definita in Cella 2)
all_image_files_in_dir = [f for f in os.listdir(IMAGE_SOURCE_DIR) if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))]

# Filtra le immagini in base ai metadati disponibili e applica il limite IMAGES_TO_PROCESS
image_files_to_process = sorted([f for f in all_image_files_in_dir if f in metadata_dict]) 

if IMAGES_TO_PROCESS is not None and IMAGES_TO_PROCESS > 0:
    image_files_to_process = image_files_to_process[:IMAGES_TO_PROCESS]
    print(f"\nLimitato a processare le prime {IMAGES_TO_PROCESS} immagini valide.")
else:
    print(f"\nProcessing tutte le {len(image_files_to_process)} immagini valide nella cartella '{IMAGE_SOURCE_DIR}'.")


if not image_files_to_process:
    print("\nATTENZIONE: Nessuna immagine trovata nella directory specificata che abbia un corrispondente nei metadati. Assicurati che i nomi dei file corrispondano tra la cartella e la colonna 'FileName' del parquet.")
    print("Processo terminato senza immagini da elaborare.")
else:
    for filename in image_files_to_process: 
        image_file_path = os.path.join(IMAGE_SOURCE_DIR, filename)
        print(f"\n--- ELABORAZIONE IMMAGINE: {filename} ({processed_images_count + 1}{f'/{len(image_files_to_process)}' if IMAGES_TO_PROCESS is not None else ''}) ---")

        # Recupero metadati per l'immagine corrente
        image_metadata = metadata_dict.get(filename, {})

        # Estrai i valori dei metadati (già puliti al caricamento del parquet)
        artwork_title = image_metadata.get('ArtworkTitle', '')
        artist_name = image_metadata.get('ArtistName', '')
        artwork_year = image_metadata.get('ArtworkYear', '')
        period = image_metadata.get('Period', '')
        style = image_metadata.get('Style', '')
        genre = image_metadata.get('Genre', '')
        movement = image_metadata.get('Movement', '')

        try:
            original_image = Image.open(image_file_path).convert("RGB")
            print(f"Immagine originale caricata: {filename} (dim: {original_image.size})")

            # --- Costruzione del contesto metadato per i prompt ---
            metadata_info_str = ""
            metadata_parts_list = [] 
            
            # Aggiungi le parti solo se il valore non è una stringa vuota o 'None' testuale (dopo la pulizia)
            if artwork_title and artwork_title.lower() not in ['none', 'sconosciuto']:
                metadata_parts_list.append(f"l'opera '{artwork_title}'")
            if artwork_year and artwork_year.lower() not in ['none', 'sconosciuto']:
                metadata_parts_list.append(f"creata nel {artwork_year}")
            if artist_name and artist_name.lower() not in ['none', 'sconosciuto']:
                metadata_parts_list.append(f"dall'artista {artist_name}")
            if style and style.lower() not in ['none', 'sconosciuto']:
                metadata_parts_list.append(f"nello stile {style}")
            if genre and genre.lower() not in ['none', 'sconosciuto']:
                metadata_parts_list.append(f"e appartenente al genere {genre}")
            if movement and movement.lower() not in ['none', 'sconosciuto']:
                metadata_parts_list.append(f"con il movimento {movement}")

            if metadata_parts_list:
                metadata_info_str = ", ".join(metadata_parts_list)
                print(f"  Contesto metadati generato per '{filename}': '{metadata_info_str}'")
            
            # Combina il prompt base con il contesto metadato (PROMPT_FRAGMENT_BASE, PROMPT_TOTAL_IMAGE_BASE, METADATA_CONTEXT_SUFFIX definiti in Cella 2)
            final_fragment_prompt = PROMPT_FRAGMENT_BASE
            if metadata_info_str:
                final_fragment_prompt += METADATA_CONTEXT_SUFFIX.format(metadata_info=metadata_info_str)
            
            final_total_prompt = PROMPT_TOTAL_IMAGE_BASE
            if metadata_info_str:
                final_total_prompt += METADATA_CONTEXT_SUFFIX.format(metadata_info=metadata_info_str)


            print(f"  Prompt finale per frammenti: {final_fragment_prompt[:100]}...")
            print(f"  Prompt finale per analisi totale: {final_total_prompt[:100]}...")
            # --- FINE Costruzione Prompt ---

            # Frammentazione in 2x2 e descrizione di ciascun segmento (GRID_ROWS, GRID_COLS definiti in Cella 2)
            segments = split_image_into_grid_segments(original_image, rows=GRID_ROWS, cols=GRID_COLS)
            print(f"  Immagine divisa in {len(segments)} segmenti (griglia {GRID_ROWS}x{GRID_COLS}).")

            # Aggiungi le descrizioni dei frammenti
            for i, segment in enumerate(segments):
                print(f"    Generazione descrizione per il segmento {i+1}...")
                base64_segment = encode_image_to_base64(segment, size=(256, 256)) # encode_image_to_base64 definita in Cella 2
                
                fragment_description = ""
                if base64_segment:
                    fragment_description = generate_description_ollama(OLLAMA_MODEL, base64_segment, final_fragment_prompt)
                    # Applica pulizia alla descrizione generata dal modello
                    # Rimuovi newline, ritorni a capo e poi pulisci gli spazi doppi
                    fragment_description = fragment_description.replace('\n', ' ').replace('\r', ' ').strip()
                    fragment_description = re.sub(r'\s+', ' ', fragment_description).strip() # Rimuovi spazi doppi

                    print(f"    Segmento {i+1} Desc: {fragment_description[:100]}...")
                else:
                    fragment_description = f"Errore: Codifica segmento {i+1} fallita."

                # Aggiungi la riga del frammento alla lista dati per il CSV
                row = {
                    "Nome Immagine": filename if i == 0 else '',
                    "ArtworkTitle": artwork_title if i == 0 else '',
                    "ArtistName": artist_name if i == 0 else '',
                    "ArtworkYear": artwork_year if i == 0 else '',
                    "Period": period if i == 0 else '',
                    "Style": style if i == 0 else '',
                    "Genre": genre if i == 0 else '',
                    "Movement": movement if i == 0 else '',
                    "Tipo Descrizione": f"Frammento {i+1}",
                    "Descrizione": fragment_description
                }
                all_image_descriptions_data.append(row)
            
            # Generazione della descrizione per l'intera immagine
            print("  Generazione descrizione per l'intera immagine...")
            base64_full_image = encode_image_to_base64(original_image, size=(512, 512))
            
            full_image_description = ""
            if base64_full_image:
                full_image_description = generate_description_ollama(OLLAMA_MODEL, base64_full_image, final_total_prompt)
                # Applica pulizia alla descrizione generata dal modello
                # Rimuovi newline, ritorni a capo e poi pulisci gli spazi doppi
                full_image_description = full_image_description.replace('\n', ' ').replace('\r', ' ').strip()
                full_image_description = re.sub(r'\s+', ' ', full_image_description).strip() # Rimuovi spazi doppi

                print(f"  Descrizione Totale: {full_image_description[:100]}...")
            else:
                full_image_description = "Errore: Codifica immagine intera fallita."

            # Aggiungi la riga della descrizione totale alla lista dati per il CSV
            row_total = {
                "Nome Immagine": '',
                "ArtworkTitle": '', "ArtistName": '', "ArtworkYear": '', "Period": '', "Style": '', "Genre": '', "Movement": '',
                "Tipo Descrizione": "Totale",
                "Descrizione": full_image_description
            }
            all_image_descriptions_data.append(row_total)
            
        except Exception as e:
            print(f"ERRORE CRITICO durante l'elaborazione dell'immagine {filename}: {e}")
            # Aggiungi righe di errore per frammenti e totale se si verifica un errore
            for i in range(TOTAL_GRID_SEGMENTS): # TOTAL_GRID_SEGMENTS definita in Cella 2
                error_row_fragment = {
                    "Nome Immagine": filename if i == 0 else '',
                    "ArtworkTitle": artwork_title if i == 0 else '',
                    "ArtistName": artist_name if i == 0 else '',
                    "ArtworkYear": artwork_year if i == 0 else '',
                    "Period": period if i == 0 else '',
                    "Style": style if i == 0 else '',
                    "Genre": genre if i == 0 else '',
                    "Movement": movement if i == 0 else '',
                    "Tipo Descrizione": f"Frammento {i+1}",
                    "Descrizione": f"ERRORE NELL'ELABORAZIONE: {e}"
                }
                all_image_descriptions_data.append(error_row_fragment)
            
            error_row_total = {
                "Nome Immagine": '',
                "ArtworkTitle": '', "ArtistName": '', "ArtworkYear": '', "Period": '', "Style": '', "Genre": '', "Movement": '',
                "Tipo Descrizione": "Totale",
                "Descrizione": f"ERRORE NELL'ELABORAZIONE: {e}"
            }
            all_image_descriptions_data.append(error_row_total)

        processed_images_count += 1
        
        # --- Ottimizzazione: Svuota la cache CUDA dopo ogni immagine ---
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print(f"  Cache CUDA svuotata dopo l'elaborazione di {filename}.")

    # --- DOPO IL LOOP PRINCIPALE, QUANDO TUTTE LE IMMMAGINI SONO STATE PROCESSATE ---

    # Definisci l'ordine delle colonne per il DataFrame finale (fondamentale per il formato)
    column_order = [
        "Nome Immagine",
        "ArtworkTitle",
        "ArtistName",
        "ArtworkYear",
        "Period",
        "Style",
        "Genre",
        "Movement",
        "Tipo Descrizione",
        "Descrizione"
    ]

    # Crea il DataFrame finale dai dati raccolti
    df_output_images = pd.DataFrame(all_image_descriptions_data, columns=column_order)

    print(f"\nProcesso di analisi immagini completato per le prime {processed_images_count} immagini con {OLLAMA_MODEL}.")
    print("DataFrame Pandas 'df_output_images' creato con successo.")

    print("\n--- Anteprima del DataFrame 'df_output_images' ---")
    # Stampa le prime 10 righe per mostrare un esempio di frammenti e analisi totale per una o più immagini
    print(df_output_images.head(10).to_markdown(index=False)) 

    # Salva il DataFrame in un file CSV SENZA la clausola 'quoting='
    # Questo userà il comportamento di quotatura predefinito di Pandas (QUOTE_MINIMAL o equivalente)
    df_output_images.to_csv(MAIN_OUTPUT_CSV_FILENAME, index=False, encoding='utf-8')
    print(f"\nDataFrame salvato come '{MAIN_OUTPUT_CSV_FILENAME}' nella stessa directory del notebook con quotatura predefinita (minimal).")

print("\nProcesso completo terminato.")



--- INIZIO ANALISI IMMAGINI con qwen2.5vl (via Ollama) ---
  MAX_TOKENS impostato a: 512

Metadati 'artgraph_metadata.parquet' caricati con successo.
Metadati preparati per la ricerca rapida tramite 'FileName' e puliti da caratteri speciali.

Processing tutte le 100 immagini valide nella cartella './images100'.

--- ELABORAZIONE IMMAGINE: aleksey-savrasov_courtyard-spring-1853.jpg (1) ---
Immagine originale caricata: aleksey-savrasov_courtyard-spring-1853.jpg (dim: (220, 275))
  Contesto metadati generato per 'aleksey-savrasov_courtyard-spring-1853.jpg': 'l'opera 'Courtyard. Spring.', creata nel 1853, dall'artista aleksey-savrasov, nello stile realism, e appartenente al genere cityscape, con il movimento Realism (arts)'
  Prompt finale per frammenti: Descrivi in modo conciso ma dettagliato **solo ciò che è visibile in questa specifica porzione dell'...
  Prompt finale per analisi totale: Analizza e descrivi l'intera immagine nel suo complesso. Focalizzati su temi generali, composizio

In [37]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import re
from transformers import CLIPProcessor, CLIPModel
import torch
import warnings # Per sopprimere alcuni warning di trasformatori se necessario

print("\n\n--- INIZIO CALCOLO STATISTICHE E ANALISI CLIP DETTAGLIATA ---")

# --- Impostazioni/Configurazione (Assicurati che queste siano allineate con le Celle 2 e 3) ---
# Se stai eseguendo questa cella isolatamente e le variabili non sono globali nel tuo ambiente:
# MAIN_OUTPUT_CSV_FILENAME = f"analisi_immagini_qwen_2x2_con_metadati_max_tokens_512.csv" # Adatta al tuo MAX_TOKENS
# IMAGE_SOURCE_DIR = './images100'
# GRID_ROWS = 2
# GRID_COLS = 2
# TOTAL_GRID_SEGMENTS = GRID_ROWS * GRID_COLS

# Ignora i FutureWarnings da transformers, spesso legati a deprecazioni future
warnings.simplefilter('ignore', FutureWarning)

# --- FUNZIONI DI UTILITY PER LE STATISTICHE DEL TESTO ---

def count_words(text: str) -> int:
    """Conta il numero di parole in una stringa, gestendo valori non stringa."""
    if not isinstance(text, str) or pd.isna(text) or text.strip() == '':
        return 0
    return len(text.split())

def tokenize_sentences(text: str) -> list[str]:
    """
    Tokenizza una stringa in frasi.
    Considera ., !, ? come terminatori di frase.
    """
    if not isinstance(text, str) or pd.isna(text) or text.strip() == '':
        return []
    # Usiamo un lookbehind (?<=[.!?]) per includere il terminatore nella frase.
    # r'\s+' cattura uno o più spazi tra le frasi.
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # Filtra eventuali stringhe vuote o solo spazi risultanti dallo split
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

def is_complete_sentence(sentence: str) -> bool:
    """Verifica se una frase è considerata completa (termina con . ! ?)."""
    if not isinstance(sentence, str) or pd.isna(sentence):
        return False
    return sentence.strip().endswith(('.', '!', '?'))

# --- FUNZIONI PER CLIP EMBEDDINGS E SIMILARITÀ ---

# Carica il modello e il processore CLIP una sola volta
clip_model = None
clip_processor = None
device = "cpu"
try:
    print("Caricamento modello CLIP (potrebbe richiedere tempo la prima volta)...")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32") # Modello CLIP comune
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    clip_model.to(device)
    print(f"Modello CLIP caricato su: {device}")
    
except Exception as e:
    print(f"ERRORE: Impossibile caricare il modello CLIP. Le metriche di similarità CLIP non saranno calcolate.")
    print(f"Dettagli errore: {e}")

def get_clip_embeddings(image: Image.Image = None, text: str = None) -> torch.Tensor:
    """
    Genera embedding CLIP per un'immagine o un testo.
    Restituisce un tensore PyTorch normalizzato.
    """
    if clip_model is None or clip_processor is None:
        return None

    try:
        if image:
            inputs = clip_processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                embeddings = clip_model.get_image_features(**inputs)
        elif text:
            inputs = clip_processor(text=str(text), return_tensors="pt", padding=True, truncation=True).to(device)
            with torch.no_grad():
                embeddings = clip_model.get_text_features(**inputs)
        else:
            return None
        
        # Normalizza gli embeddings per il calcolo della similarità coseno
        return embeddings / embeddings.norm(p=2, dim=-1, keepdim=True)
    except Exception as e:
        # print(f"Avviso: Errore nella generazione embedding CLIP ({'immagine' if image else 'testo'}): {e}")
        return None

def calculate_cosine_similarity(embedding1: torch.Tensor, embedding2: torch.Tensor) -> float:
    """
    Calcola la similarità coseno tra due embedding normalizzati.
    """
    if embedding1 is None or embedding2 is None:
        return np.nan
    try:
        # Assicurati che gli embedding abbiano la stessa dimensione e non siano tensori 0-dimensionali
        if embedding1.ndim == 0: embedding1 = embedding1.unsqueeze(0)
        if embedding2.ndim == 0: embedding2 = embedding2.unsqueeze(0)
        
        return torch.dot(embedding1.squeeze(), embedding2.squeeze()).item()
    except Exception as e:
        # print(f"Avviso: Errore nel calcolo similarità coseno: {e}")
        return np.nan

# --- FUNZIONE PER DIVIDERE IMMAGINE (Duplicata per autonomia, ma preferibile importarla se il notebook lo permette) ---
def split_image_into_grid_segments(image: Image.Image, rows: int, cols: int) -> list[Image.Image]:
    """
    Divide un'immagine PIL in una griglia di segmenti (e.g., 2x2).
    Restituisce i segmenti in ordine di lettura (da sinistra a destra, dall'alto in basso).
    """
    width, height = image.size
    segment_width = width // cols
    segment_height = height // rows
    segments = []
    
    for r in range(rows):
        for c in range(cols):
            left = c * segment_width
            upper = r * segment_height
            # Gestione dei bordi per assicurarsi che tutti i pixel siano inclusi
            right = (c + 1) * segment_width if c < cols - 1 else width
            lower = (r + 1) * segment_height if r < rows - 1 else height
            
            segment = image.crop((left, upper, right, lower))
            segments.append(segment)
            
    return segments

# --- CARICAMENTO E PREPARAZIONE DATAFRAME ---
df_output = None
try:
    df_output = pd.read_csv(MAIN_OUTPUT_CSV_FILENAME)
    print(f"\nDataFrame '{MAIN_OUTPUT_CSV_FILENAME}' caricato con successo per l'analisi.")
    print(f"Numero di righe nel DataFrame caricato: {len(df_output)}")
except FileNotFoundError:
    print(f"ERRORE: File '{MAIN_OUTPUT_CSV_FILENAME}' non trovato. Assicurati di aver eseguito la Cella 3.")
    exit()
except Exception as e:
    print(f"ERRORE nel caricamento del DataFrame: {e}")
    exit()

# Assicurati che le colonne chiave esistano
required_columns = ["Nome Immagine", "Tipo Descrizione", "Descrizione"]
if not all(col in df_output.columns for col in required_columns):
    print(f"ERRORE: Il DataFrame caricato non contiene tutte le colonne richieste: {required_columns}")
    exit()

# --- RACCOLTA DATI PER STATISTICHE ---
all_total_lengths = []
all_fragment_lengths = []

all_total_sentence_lengths = []
total_complete_sentences = 0
total_incomplete_sentences = 0

all_fragment_sentence_lengths = [] # NUOVA LISTA PER LUNGHEZZE FRASI FRAMMENTI
fragment_complete_sentences = 0    # NUOVO CONTATORE PER FRASI COMPLETE FRAMMENTI
fragment_incomplete_sentences = 0  # NUOVO CONTATORE PER FRASI INCOMPLETE FRAMMENTI

all_clip_similarities_total = []
all_clip_similarities_fragments = []

print("\nAvvio calcolo statistiche e similarità CLIP per ogni immagine...")

# Ottieni i nomi delle immagini uniche che iniziano un blocco (non stringhe vuote)
unique_image_filenames = df_output['Nome Immagine'].dropna().unique().tolist()
# Filtra ulteriormente per rimuovere stringhe vuote se ce ne fossero dopo dropna (improbabile, ma per sicurezza)
unique_image_filenames = [f for f in unique_image_filenames if f.strip() != '']

print(f"Trovate {len(unique_image_filenames)} immagini uniche da analizzare.")

for filename in unique_image_filenames:
    print(f"Analizzando i dati per l'immagine: {filename}")
    
    # Seleziona tutte le righe relative all'immagine corrente (inclusi i frammenti e il totale)
    start_index = df_output[df_output['Nome Immagine'] == filename].index[0]
    next_image_start_indices = df_output.loc[start_index + 1:, 'Nome Immagine'].replace('', np.nan).dropna().index
    
    end_index = len(df_output) 
    if not next_image_start_indices.empty:
        end_index = next_image_start_indices[0]
        
    current_image_block_df = df_output.iloc[start_index:end_index].copy()

    # Recupera la descrizione totale
    total_description_series = current_image_block_df[current_image_block_df['Tipo Descrizione'] == 'Totale']['Descrizione']
    total_description = str(total_description_series.iloc[0]) if not total_description_series.empty else ''
    
    # Recupera le descrizioni dei frammenti
    fragment_descriptions_current_image = []
    for i in range(1, TOTAL_GRID_SEGMENTS + 1):
        fragment_desc_series = current_image_block_df[current_image_block_df['Tipo Descrizione'] == f'Frammento {i}']['Descrizione']
        fragment_description = str(fragment_desc_series.iloc[0]) if not fragment_desc_series.empty else ''
        fragment_descriptions_current_image.append(fragment_description)

    # --- Calcolo Statistiche di Lunghezza (Parole) ---
    total_word_count = count_words(total_description)
    all_total_lengths.append(total_word_count)
    print(f"  Descrizione Totale '{filename}' - Parole: {total_word_count}")

    for fragment_description in fragment_descriptions_current_image:
        fragment_word_count = count_words(fragment_description)
        all_fragment_lengths.append(fragment_word_count)
    # print(f"  Descrizioni Frammenti '{filename}' - Parole Medie: {np.mean([count_words(d) for d in fragment_descriptions_current_image]):.2f}") # Meno verbose in loop

    # --- Calcolo Statistiche Frasi per Descrizioni Totali ---
    sentences = tokenize_sentences(total_description)
    for sentence in sentences:
        all_total_sentence_lengths.append(count_words(sentence))
        if is_complete_sentence(sentence):
            total_complete_sentences += 1
        else:
            total_incomplete_sentences += 1
    # if not sentences: # Meno verbose in loop
    #     print(f"  Avviso: Nessuna frase rilevata nella descrizione totale di '{filename}'.")

    # --- Calcolo Statistiche Frasi per Descrizioni Frammenti (NUOVA SEZIONE) ---
    for fragment_description in fragment_descriptions_current_image:
        fragment_sentences = tokenize_sentences(fragment_description)
        for sentence in fragment_sentences:
            all_fragment_sentence_lengths.append(count_words(sentence))
            if is_complete_sentence(sentence):
                fragment_complete_sentences += 1
            else:
                fragment_incomplete_sentences += 1
        # if not fragment_sentences: # Meno verbose in loop
        #     print(f"  Avviso: Nessuna frase rilevata nella descrizione di un frammento di '{filename}'.")


    # --- Calcolo Similarità CLIP ---
    if clip_model and clip_processor:
        image_file_path = os.path.join(IMAGE_SOURCE_DIR, filename)
        if os.path.exists(image_file_path):
            try:
                original_image = Image.open(image_file_path).convert("RGB")
                
                # Similarità: Immagine Totale vs. Descrizione Totale
                img_total_embedding = get_clip_embeddings(image=original_image)
                text_total_embedding = get_clip_embeddings(text=total_description)
                similarity_total = calculate_cosine_similarity(img_total_embedding, text_total_embedding)
                all_clip_similarities_total.append(similarity_total)
                print(f"  Similarità CLIP Totale '{filename}': {similarity_total:.4f}")

                # Similarità: Frammenti Immagine vs. Descrizioni Frammenti
                segments_images = split_image_into_grid_segments(original_image, rows=GRID_ROWS, cols=GRID_COLS)
                for i, segment_image in enumerate(segments_images):
                    if i < len(fragment_descriptions_current_image):
                        fragment_description_text = fragment_descriptions_current_image[i]
                        img_fragment_embedding = get_clip_embeddings(image=segment_image)
                        text_fragment_embedding = get_clip_embeddings(text=fragment_description_text)
                        similarity_fragment = calculate_cosine_similarity(img_fragment_embedding, text_fragment_embedding)
                        all_clip_similarities_fragments.append(similarity_fragment)
                    else:
                        all_clip_similarities_fragments.append(np.nan) 
            except Exception as e:
                print(f"  ERRORE CLIP per l'immagine '{filename}': {e}. Similarità non calcolate.")
                all_clip_similarities_total.append(np.nan)
                for _ in range(TOTAL_GRID_SEGMENTS):
                    all_clip_similarities_fragments.append(np.nan)
        else:
            print(f"  AVVISO: Immagine '{filename}' non trovata in '{IMAGE_SOURCE_DIR}'. Similarità CLIP non calcolate.")
            all_clip_similarities_total.append(np.nan)
            for _ in range(TOTAL_GRID_SEGMENTS):
                all_clip_similarities_fragments.append(np.nan)
    else:
        print("  CLIP Model/Processor non disponibile. Similarità CLIP non calcolate.")
        all_clip_similarities_total.append(np.nan)
        for _ in range(TOTAL_GRID_SEGMENTS):
            all_clip_similarities_fragments.append(np.nan)
            
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        # print(f"  Cache CUDA svuotata dopo l'analisi CLIP di {filename}.")

print("\n--- RISULTATI STATISTICI FINALI ---")

# --- Funzione helper per stampare statistiche ---
def print_stats(name, data_list):
    if data_list and any(~np.isnan(data_list)):
        valid_data = [d for d in data_list if not np.isnan(d)]
        if valid_data:
            print(f"\n--- Statistiche {name} ---")
            print(f"  Media: {np.mean(valid_data):.2f}")
            print(f"  Minima: {np.min(valid_data):.2f}")
            print(f"  Massima: {np.max(valid_data):.2f}")
            print(f"  Conteggio Validi: {len(valid_data)}")
        else:
            print(f"\n--- Statistiche {name} ---")
            print("  Nessun valore valido per il calcolo.")
    else:
        print(f"\n--- Statistiche {name} ---")
        print("  Lista vuota o solo NaN per il calcolo.")


# --- STAMPA E SALVATAGGIO ---
print_stats("Lunghezza Descrizioni Totali (Parole)", all_total_lengths)
print_stats("Lunghezza Descrizioni Frammenti (Parole)", all_fragment_lengths)

print("\n--- Statistiche Frasi (Descrizioni Totali) ---")
if all_total_sentence_lengths:
    avg_sentence_length_total = np.mean(all_total_sentence_lengths)
    print(f"  Lunghezza media delle frasi (parole): {avg_sentence_length_total:.2f}")

    sentences_below_avg_total = sum(1 for l in all_total_sentence_lengths if l < avg_sentence_length_total)
    sentences_above_avg_total = sum(1 for l in all_total_sentence_lengths if l > avg_sentence_length_total)
    sentences_equal_avg_total = sum(1 for l in all_total_sentence_lengths if l == avg_sentence_length_total)
    
    print(f"  Frasi con lunghezza < media: {sentences_below_avg_total}")
    print(f"  Frasi con lunghezza = media: {sentences_equal_avg_total}")
    print(f"  Frasi con lunghezza > media: {sentences_above_avg_total}")
    print(f"  Numero totale di frasi: {len(all_total_sentence_lengths)}")
    print(f"  Frasi complete: {total_complete_sentences}")
    print(f"  Frasi incomplete: {total_incomplete_sentences}")
else:
    print("  Nessuna frase rilevata per le descrizioni totali.")

# NUOVE STATISTICHE FRASI PER FRAMMENTI
print("\n--- Statistiche Frasi (Descrizioni Frammenti) ---")
if all_fragment_sentence_lengths:
    avg_sentence_length_fragment = np.mean(all_fragment_sentence_lengths)
    print(f"  Lunghezza media delle frasi (parole): {avg_sentence_length_fragment:.2f}")

    sentences_below_avg_fragment = sum(1 for l in all_fragment_sentence_lengths if l < avg_sentence_length_fragment)
    sentences_above_avg_fragment = sum(1 for l in all_fragment_sentence_lengths if l > avg_sentence_length_fragment)
    sentences_equal_avg_fragment = sum(1 for l in all_fragment_sentence_lengths if l == avg_sentence_length_fragment)
    
    print(f"  Frasi con lunghezza < media: {sentences_below_avg_fragment}")
    print(f"  Frasi con lunghezza = media: {sentences_equal_avg_fragment}")
    print(f"  Frasi con lunghezza > media: {sentences_above_avg_fragment}")
    print(f"  Numero totale di frasi: {len(all_fragment_sentence_lengths)}")
    print(f"  Frasi complete: {fragment_complete_sentences}")
    print(f"  Frasi incomplete: {fragment_incomplete_sentences}")
else:
    print("  Nessuna frase rilevata per le descrizioni dei frammenti.")


# Utilizza la funzione print_stats per le similarità CLIP
print_stats("Similarità CLIP (Immagine Totale vs. Descrizione Totale)", all_clip_similarities_total)
print_stats("Similarità CLIP (Frammento Immagine vs. Descrizione Frammento)", all_clip_similarities_fragments)


# --- SALVATAGGIO STATISTICHE IN CSV ---
stats_data = {
    'Metriche': [
        'Lunghezza Media Descrizioni Totali (Parole)', 'Lunghezza Minima Descrizioni Totali (Parole)', 'Lunghezza Massima Descrizioni Totali (Parole)',
        'Lunghezza Media Descrizioni Frammenti (Parole)', 'Lunghezza Minima Descrizioni Frammenti (Parole)', 'Lunghezza Massima Descrizioni Frammenti (Parole)',
        'Lunghezza Media Frasi Totali (Parole)',
        'Frasi < Media (Totali)', 'Frasi = Media (Totali)', 'Frasi > Media (Totali)',
        'Frasi Complete (Totali)', 'Frasi Incomplete (Totali)',
        'Lunghezza Media Frasi Frammenti (Parole)', # NUOVA VOCE
        'Frasi < Media (Frammenti)', # NUOVA VOCE
        'Frasi = Media (Frammenti)', # NUOVA VOCE
        'Frasi > Media (Frammenti)', # NUOVA VOCE
        'Frasi Complete (Frammenti)', # NUOVA VOCE
        'Frasi Incomplete (Frammenti)', # NUOVA VOCE
        'Similarità Media CLIP (Totale)', 'Similarità Minima CLIP (Totale)', 'Similarità Massima CLIP (Totale)',
        'Similarità Media CLIP (Frammenti)', 'Similarità Minima CLIP (Frammenti)', 'Similarità Massima CLIP (Frammenti)'
    ],
    'Valore': [
        np.mean(all_total_lengths) if all_total_lengths else np.nan,
        np.min(all_total_lengths) if all_total_lengths else np.nan,
        np.max(all_total_lengths) if all_total_lengths else np.nan,
        np.mean(all_fragment_lengths) if all_fragment_lengths else np.nan,
        np.min(all_fragment_lengths) if all_fragment_lengths else np.nan,
        np.max(all_fragment_lengths) if all_fragment_lengths else np.nan,
        np.mean(all_total_sentence_lengths) if all_total_sentence_lengths else np.nan,
        sentences_below_avg_total if all_total_sentence_lengths else np.nan,
        sentences_equal_avg_total if all_total_sentence_lengths else np.nan,
        sentences_above_avg_total if all_total_sentence_lengths else np.nan,
        total_complete_sentences,
        total_incomplete_sentences,
        np.mean(all_fragment_sentence_lengths) if all_fragment_sentence_lengths else np.nan, # NUOVO VALORE
        sentences_below_avg_fragment if all_fragment_sentence_lengths else np.nan,          # NUOVO VALORE
        sentences_equal_avg_fragment if all_fragment_sentence_lengths else np.nan,           # NUOVO VALORE
        sentences_above_avg_fragment if all_fragment_sentence_lengths else np.nan,           # NUOVO VALORE
        fragment_complete_sentences,                                                         # NUOVO VALORE
        fragment_incomplete_sentences,                                                       # NUOVO VALORE
        np.mean([s for s in all_clip_similarities_total if not np.isnan(s)]) if all_clip_similarities_total and any(~np.isnan(all_clip_similarities_total)) else np.nan,
        np.min([s for s in all_clip_similarities_total if not np.isnan(s)]) if all_clip_similarities_total and any(~np.isnan(all_clip_similarities_total)) else np.nan,
        np.max([s for s in all_clip_similarities_total if not np.isnan(s)]) if all_clip_similarities_total and any(~np.isnan(all_clip_similarities_total)) else np.nan,
        np.mean([s for s in all_clip_similarities_fragments if not np.isnan(s)]) if all_clip_similarities_fragments and any(~np.isnan(all_clip_similarities_fragments)) else np.nan,
        np.min([s for s in all_clip_similarities_fragments if not np.isnan(s)]) if all_clip_similarities_fragments and any(~np.isnan(all_clip_similarities_fragments)) else np.nan,
        np.max([s for s in all_clip_similarities_fragments if not np.isnan(s)]) if all_clip_similarities_fragments and any(~np.isnan(all_clip_similarities_fragments)) else np.nan
    ]
}

df_stats = pd.DataFrame(stats_data)
stats_output_csv_filename = f"statistiche_descrizioni_max_tokens_{MAX_TOKENS}.csv"
df_stats.to_csv(stats_output_csv_filename, index=False, encoding='utf-8')
print(f"\nStatistiche riassuntive salvate come '{stats_output_csv_filename}'.")

print("\nProcesso di calcolo statistiche e analisi CLIP terminato.")



--- INIZIO CALCOLO STATISTICHE E ANALISI CLIP DETTAGLIATA ---
Caricamento modello CLIP (potrebbe richiedere tempo la prima volta)...
Modello CLIP caricato su: cpu

DataFrame 'analisi_immagini_qwen_2x2_con_metadati_max_tokens_512.csv' caricato con successo per l'analisi.
Numero di righe nel DataFrame caricato: 500

Avvio calcolo statistiche e similarità CLIP per ogni immagine...
Trovate 100 immagini uniche da analizzare.
Analizzando i dati per l'immagine: aleksey-savrasov_courtyard-spring-1853.jpg
  Descrizione Totale 'aleksey-savrasov_courtyard-spring-1853.jpg' - Parole: 284
  Similarità CLIP Totale 'aleksey-savrasov_courtyard-spring-1853.jpg': 0.2761
Analizzando i dati per l'immagine: alfred-sisley_village-on-the-banks-of-the-seine-1872.jpg
  Descrizione Totale 'alfred-sisley_village-on-the-banks-of-the-seine-1872.jpg' - Parole: 260
  Similarità CLIP Totale 'alfred-sisley_village-on-the-banks-of-the-seine-1872.jpg': 0.3581
Analizzando i dati per l'immagine: chaldin-alex_lilies.jpg
 